In [2]:
#import keras
#from keras.application import vgg16
import mysql.connector
from mysql.connector import errorcode

config = {
    'user': 'root',
    'password': 'GETFILE',
    'host': 'GETFILE',
    'database': 'usr'
}

transaction_state = {
    'user_id': 0,
    'loc_id': 0,
    'start_time': 0,
    'valid': 0
}

THIS_LOCATION = 1

from PIL import Image, ImageDraw, ImageFont
from matplotlib.pyplot import imshow
import pytesseract, cv2
import numpy as np
import pytesseract
import time
import pyodbc
import datetime
%matplotlib inline

In [3]:
dat = open('passw', 'r').read().split()
config['host'] = dat[0]
config['password'] = dat[1]
config

{'user': 'root',
 'password': 'toor',
 'host': '35.193.20.163',
 'database': 'usr'}

In [4]:
def test(s):
    img = Image.open(s)
    return pytesseract.image_to_string(s)

def testrun():
    return test('license.jpg')

In [5]:
def processImage(img, exp = 112):
    w, h = img.size
    #x, y, wcrop, hcrop = 120, 78, 50, 50
    cropped_image = img#.crop((x, y, w - wcrop, h - hcrop))
    gray = cropped_image.convert('L')
    bw = gray.point(lambda x: 0 if x<exp else 255, '1')
    cropped_image = bw.convert('RGBA')
    return cropped_image
    
def show(i):
    imshow(np.asarray(i))

In [ ]:
img = Image.open('license.jpg').convert('RGBA')
w, h = img.size

In [ ]:
x, y, wcrop, hcrop = 120, 78, 50, 50
cropped_image = img.crop((x, y, w - wcrop, h - hcrop))
#show(cropped_image)
gray = cropped_image.convert('L')
bw = gray.point(lambda x: 0 if x<38 else 255, '1')
show(bw.convert('RGBA'))
cropped_image = bw.convert('RGBA')

In [ ]:
boxes = pytesseract.image_to_boxes(cropped_image)
pytesseract.image_to_string(cropped_image)

In [ ]:
bout = Image.new('RGBA', cropped_image.size, (255,255,255,0))
d = ImageDraw.Draw(bout)
#[[_[0]] + [int(__) for __ in _.split()[1:]] for _ in boxes.split('\n')[1:]]
[d.rectangle([int(__) for __ in _.split()[1:-1]]) for _ in boxes.split('\n')[:]]
d.text((100, 10), "Hello", fill=(255, 0, 0, 255))

In [ ]:
out = Image.alpha_composite(cropped_image, bout)
show(out)

In [6]:
def get_plate():
    refresh_rate = 750
    guesses, best_guess = {}, ""
    gcount, gmax = 0, 25
    fnt = ImageFont.truetype('Pillow/Tests/fonts/FreeMono.ttf', 40)
    x, t = 0, time.clock()
    cap = cv2.VideoCapture (0)
    while (True):
        ret, frame = cap.read()
        if not ret:
            continue
        gray = cv2.cvtColor (frame, cv2.COLOR_BGR2GRAY)
        gray = processImage(Image.fromarray(gray))

        if x%refresh_rate:
            txt = pytesseract.image_to_string(gray)
            guesses.update({txt: guesses[txt] + 1 if txt in guesses else 1})
            if txt is not '':
                gcount += 1

            bout = Image.new('RGBA', gray.size, (255,255,255,0))
            bw, bh = bout.size
            d = ImageDraw.Draw(bout)
            d.text((10,10), ascii(txt), font=fnt, fill=(255,0,0,255))

            best_guess = make_guess(guesses)
            if valid_guess(best_guess):
                d.text((10, bh - 50), best_guess, font=fnt, fill=(0,255,0,255))
            else:
                d.text((10, bh - 50), best_guess, font=fnt, fill=(255,0,0,255))
            gray = Image.alpha_composite(gray, bout)

            #print(1/(time.clock() - t))
            t = time.clock()
            if gcount > gmax:
                gcount = 0
                guesses = {}

        cv2.imshow('frame', np.array(gray.convert('RGB'))[:, :, ::-1].copy())
        if cv2.waitKey (1) & 0xFF == ord('q'):
            return best_guess
        x += 1
        if len(best_guess) > 1 and valid_guess(best_guess):
            return best_guess

In [7]:
def make_guess(g):
    km, vm = '', 0
    for k, v in g.items():
        if k is not '' and v > 1:
            if v > vm:
                vm = v
                km = k
    return km

def valid_guess(s):
    domain = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-'
    for _ in s:
        if _ not in domain:
            return False
    return True

def makeSQLtime():
    return int(time.ctime().split()[3].replace(':','')), str(datetime.datetime.today().date())

In [154]:
x, y = makeSQLtime()
y

'2019-03-23'

In [12]:
def createTransaction():
    global transaction_state
    inf = []
    guess = get_plate()
    cnx = cur = None
    try:
        cnx = mysql.connector.connect(**config)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print('Something is wrong with your user name or password')
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
    else:
        cur = cnx.cursor()
        #cur.execute('show databases;')
        #for row in cur.fetchall():
        #    print(row)

        cur.execute('USE usr;')
        cur.execute('SELECT * FROM user WHERE plate_num = "{}";'.format(guess))
        for row in cur.fetchall():
            inf += [row]
            print(row)

        if len(inf) is 1:
            START_TIME, START_DATE = makeSQLtime()
            cur.execute('INSERT INTO transaction (user_id, loc_id, start_time, start_date, price) VALUES ({}, {}, {}, "{}", {});'.format(
                inf[0][0], 
                THIS_LOCATION, 
                START_TIME, 
                START_DATE, 
                getLocationRate(THIS_LOCATION)))
            cnx.commit()
            transaction_state = {
                'user_id': inf[0][0],
                'loc_id': THIS_LOCATION,
                'start_time': START_TIME,
                'start_date': START_DATE,
                'valid': 1
            }

            print('Transaction created')
        else:
            print('Invalid user selection!')
            print('Prompt for user information on panel')
    finally:
        if cur:
            cur.close()
        if cnx:
            cnx.close()

In [12]:
def closeTransaction():
    global transaction_state
    if transaction_state['valid'] is 0:
        return
    
    inf = []
    print(transaction_state)
    cnx = cur = None
    try:
        cnx = mysql.connector.connect(**config)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print('Something is wrong with your user name or password')
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
    else:
        cur = cnx.cursor()

        cur.execute('USE usr;')
        cur.execute('SELECT * FROM transaction WHERE user_id = {} AND loc_id = {} AND start_time = {} AND start_date = "{}";'.format(
            transaction_state['user_id'],
            transaction_state['loc_id'],
            transaction_state['start_time'],
            transaction_state['start_date']
        ))
        for row in cur.fetchall():
            inf += [row]
            print(row)

        if len(inf) is 1:
            END_TIME, END_DATE = makeSQLtime()
            cur.execute('UPDATE transaction SET end_time = {}, end_date = "{}"  WHERE user_id = {} AND loc_id = {} AND start_time = {} AND start_date = "{}";'.format(
                END_TIME,
                END_DATE,
                transaction_state['user_id'],
                THIS_LOCATION,
                transaction_state['start_time'],
                transaction_state['start_date']
            ))
            cnx.commit()
            transaction_state = {
                'user_id': 0,
                'loc_id': 0,
                'start_time': 0,
                'valid': 0
            }

            print('Transaction updated')
        else:
            print('Invalid user selection!')
            print('Prompt for user information on panel')
    finally:
        if cur:
            cur.close()
        if cnx:
            cnx.close()
  

In [15]:
def getLocationRate(loc_id):
    dat = []
    cnx = cur = None
    try:
        cnx = mysql.connector.connect(**config)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print('Something is wrong with your user name or password')
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
    else:
        cur = cnx.cursor()

        cur.execute('USE usr;')
        cur.execute('SELECT cost FROM rate WHERE loc_id = {};'.format(loc_id))
        for row in cur.fetchall():
            dat += [row]
            #print(row)
    finally:
        if cur:
            cur.close()
        if cnx:
            cnx.close()
    return dat[0][0]

In [12]:
def getCostByLicense(license):
    dat = []
    cnx = cur = None
    try:
        cnx = mysql.connector.connect(**config)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print('Something is wrong with your user name or password')
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)
    else:
        cur = cnx.cursor()

        cur.execute('USE usr;')
        cur.execute('SELECT trans_id, loc_id, start_date, start_time, MINUTE(TIMEDIFF(cast(concat(end_date, " ", end_time) as datetime), cast(concat(start_date, " ", start_time) as datetime)))*Price AS Cost FROM user NATURAL JOIN transaction WHERE plate_num = "{}";'.format(license))
        for row in cur.fetchall():
            dat += [row]
            #print(row)
    finally:
        if cur:
            cur.close()
        if cnx:
            cnx.close()
    return dat

In [ ]:
createTransaction()

In [18]:
closeTransaction()

{'user_id': 4, 'loc_id': 1, 'start_time': 230024, 'start_date': '2019-03-23', 'valid': 1}
(2, 4, 1, datetime.timedelta(0, 82824), datetime.date(2019, 3, 23), None, None, 10)
Transaction updated


In [20]:
d = getCostByLicense('MFA-81')
row_titles = [["Transaction ID", "Location ID", "Start Date", "Start Time", "Total Cost"]]
[[print("{:>20}".format(str(__)), end = '\t') for __ in _] + [print()] for _ in row_titles + d];None

      Transaction ID	         Location ID	          Start Date	          Start Time	          Total Cost	
                   2	                   1	          2019-03-23	            23:00:24	                  60	


In [141]:
datetime.datetime.today()

23